# Security Company Scheduler V6

This notebook implements a scheduling system for a security company using genetic algorithms.

In [11]:
import datetime
import pandas as pd
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from constants import AGENTS, APPOINTMENT_TYPES
from models import Agent, Meeting
from meeting_generator import generate_meetings
from improved_genetic_algorithm import run_scheduling_algorithm  # Updated import
from visualization import create_schedule_dataframe, plot_schedule, plot_schedule_statistics, analyze_schedule, plot_fitness_history

## Set Parameters

In [12]:
@interact
def set_parameters(num_agents=widgets.IntSlider(min=1, max=10, step=1, value=5),
                   num_days=widgets.IntSlider(min=1, max=14, step=1, value=7),
                   num_clients=widgets.IntSlider(min=1, max=20, step=1, value=5),
                   population_size=widgets.IntSlider(min=10, max=200, step=10, value=50),
                   num_generations=widgets.IntSlider(min=10, max=500, step=10, value=100)):
    global PARAMS
    PARAMS = {
        'num_agents': num_agents,
        'num_days': num_days,
        'num_clients': num_clients,
        'population_size': population_size,
        'num_generations': num_generations
    }
    print("Parameters set successfully.")

interactive(children=(IntSlider(value=5, description='num_agents', max=10, min=1), IntSlider(value=7, descript…

## Generate Meetings

In [13]:
def generate_and_display_meetings():
    start_date = datetime.date.today()
    global meetings
    meetings = generate_meetings(start_date, num_clients=PARAMS['num_clients'], days=PARAMS['num_days'])
    
    meetings_df = pd.DataFrame([
        {
            'Start': m.start,
            'End': m.end,
            'Type': m.required_skill,
            'Is Night': m.is_night,
            'Duration (hours)': (m.end - m.start).total_seconds() / 3600
        } for m in meetings
    ])
    
    print(f"Generated {len(meetings)} meetings")
    display(meetings_df)
    
    # Create a calendar view of the meetings
    fig = px.timeline(meetings_df, x_start="Start", x_end="End", y="Type", color="Is Night",
                      title="Generated Meetings Calendar")
    fig.update_yaxes(categoryorder="category ascending")
    
    # Add vertical lines for day boundaries
    for day in pd.date_range(meetings_df['Start'].min(), meetings_df['End'].max()):
        fig.add_vline(x=day.replace(hour=5, minute=0), line_dash="dash", line_color="gray")
    
    # Update x-axis to show date on bottom and time on top
    fig.update_xaxes(
        tickformat="%H:%M",
        tickangle=0,
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='lightgray'
    )
    fig.update_layout(
        xaxis=dict(
            rangeslider=dict(visible=False),
            tickformat="%Y-%m-%d"
        ),
        xaxis2=dict(
            overlaying="x",
            side="top",
            tickformat="%H:%M",
            showgrid=False
        )
    )
    
    fig.show()
    
    return meetings_df

interact_manual(generate_and_display_meetings)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.generate_and_display_meetings()>

## Run the Scheduling Algorithm

In [14]:
def run_algorithm():
    global agents, meetings, schedule_df, fitness_history
    if 'meetings' not in globals() or meetings is None:
        print("Please generate meetings first.")
        return

    # Create agent objects
    agents = [Agent(f"Agent{i+1}", AGENTS[f"Agent{i+1}"]) for i in range(PARAMS['num_agents'])]

    print(f"Running algorithm with {len(agents)} agents and {len(meetings)} meetings")

    # Run the improved scheduling algorithm
    final_schedule, fitness_history = run_scheduling_algorithm(agents, meetings, 
                                                               pop_size=PARAMS['population_size'], 
                                                               generations=PARAMS['num_generations'])

    print(f"Scheduled {len(final_schedule)} meetings")
    print(f"Final fitness score: {fitness_history[-1]}")
    
    # Create and display schedule dataframe
    schedule_df = create_schedule_dataframe(final_schedule)
    display(schedule_df)
    
    # Plot fitness history
    fig = plot_fitness_history(fitness_history)
    fig.show()
    
    # Analyze the schedule
    analyze_schedule(schedule_df)
    
    return schedule_df

interact_manual(run_algorithm)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.run_algorithm()>

## Visualize the Results

In [15]:
def visualize_results():
    global schedule_df
    if 'schedule_df' not in globals() or schedule_df is None:
        print("Please run the scheduling algorithm first to generate a schedule.")
        return
    
    fig = plot_schedule(schedule_df)
    fig.show()

    stats_fig = plot_schedule_statistics(schedule_df)
    stats_fig.show()

interact_manual(visualize_results)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.visualize_results()>

## Analyze the Schedule

In [16]:
def analyze_results():
    global schedule_df
    if 'schedule_df' not in globals() or schedule_df is None:
        print("Please run the scheduling algorithm first to generate a schedule.")
        return
    
    analyze_schedule(schedule_df)

interact_manual(analyze_results)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function __main__.analyze_results()>

In [17]:
# Genetic Algorithm Explanation

# Run the genetic algorithm
best_schedule, fitness_history = run_scheduling_algorithm(agents, meetings)

# Plot the fitness history
fig = plot_fitness_history(fitness_history)
fig.show()

print("Genetic Algorithm Process Explanation:")
print("1. Initialization:")
print("   - We start with a population of random schedules (individuals).")
print("   - Each schedule is a mapping of meetings to agents.")
print()
print("2. Fitness Evaluation:")
print("   - We evaluate each schedule based on various criteria:")
print("     a) Skill matching")
print("     b) Working hours (aiming for 6-10 hours per day)")
print("     c) Even distribution of total hours among agents")
print("     d) Even distribution of night shifts")
print("     e) Proper breaks between meetings")
print()
print("3. Selection:")
print("   - We select the best schedules to become parents for the next generation.")
print("   - We use tournament selection: randomly choose a few schedules and select the best among them.")
print()
print("4. Crossover:")
print("   - We combine two parent schedules to create child schedules.")
print("   - We use two-point crossover: choose two random points and exchange the middle section between parents.")
print()
print("5. Mutation:")
print("   - We introduce small random changes to some schedules.")
print("   - For each meeting in a schedule, there's a small chance we'll reassign it to a different eligible agent.")
print()

Genetic Algorithm Process Explanation:
1. Initialization:
   - We start with a population of random schedules (individuals).
   - Each schedule is a mapping of meetings to agents.

2. Fitness Evaluation:
   - We evaluate each schedule based on various criteria:
     a) Skill matching
     b) Working hours (aiming for 6-10 hours per day)
     c) Even distribution of total hours among agents
     d) Even distribution of night shifts
     e) Proper breaks between meetings

3. Selection:
   - We select the best schedules to become parents for the next generation.
   - We use tournament selection: randomly choose a few schedules and select the best among them.

4. Crossover:
   - We combine two parent schedules to create child schedules.
   - We use two-point crossover: choose two random points and exchange the middle section between parents.

5. Mutation:
   - We introduce small random changes to some schedules.
   - For each meeting in a schedule, there's a small chance we'll reassign it 